In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!git clone https://github.com/ItWasAllYellow/public_cs224n_gpt.git

%cd /content/public_cs224n_gpt

Cloning into 'public_cs224n_gpt'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 69 (delta 21), reused 13 (delta 13), pack-reused 32 (from 1)
Receiving objects: 100% (69/69), 30.87 MiB | 29.66 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/public_cs224n_gpt


In [3]:
"""
train_expl_all_curr.py
GPT-2 해설 생성 (모든 선택지 포함) + Curriculum Learning (grade 2 ➜ grade 3)
"""

import argparse, os, random, numpy as np, pandas as pd, torch
import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model   # 반드시 여러분의 커스텀 GPT2Model

TQDM_DISABLE = False  # True면 TQDM bar 숨김

# ───────────────────────────── utils
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False

# ───────────────────────────── 모델
class GPT2Explainer(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.backbone = GPT2Model.from_pretrained(
            model=args.model_size, d=args.d, l=args.l, num_heads=args.num_heads
        )
        vocab = GPT2Tokenizer.from_pretrained(args.model_size).vocab_size
        self.lm_head = nn.Linear(args.d, vocab, bias=False)
        for p in self.parameters(): p.requires_grad = True

    def forward(self, ids, mask, labels=None):
        h = self.backbone(ids, attention_mask=mask)["last_hidden_state"]
        logits = self.lm_head(h)                       # (B, L, V)
        if labels is None:
            return logits
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=-100
        )
        return loss, logits

# ───────────────────────────── 데이터셋
class ExplDataset(Dataset):
    def __init__(self, df, tok, max_len=512):
        self.df, self.tok, self.max_len = df.reset_index(drop=True), tok, max_len
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        return r["input"], r["target"]
    def collate_fn(self, batch):
        ins, tgs = zip(*batch)
        enc_in  = self.tok(list(ins), padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        enc_out = self.tok(list(tgs), padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        labels = enc_out.input_ids.clone()
        labels[enc_out.attention_mask == 0] = -100
        return {"input_ids": enc_in.input_ids,
                "attention_mask": enc_in.attention_mask,
                "labels": labels}

# ───────────────────────────── 평가
def evaluate(loader, model, dev):
    model.eval(); tot_loss = tok_cnt = 0
    with torch.no_grad():
        for bt in loader:
            ids  = bt["input_ids"].to(dev)
            mask = bt["attention_mask"].to(dev)
            lbls = bt["labels"].to(dev)
            loss, _ = model(ids, mask, lbls)
            num_tok = (lbls != -100).sum().item()
            tot_loss += loss.item() * num_tok
            tok_cnt  += num_tok
    ppl = np.exp(tot_loss / max(tok_cnt, 1))
    return ppl

# ───────────────────────────── 학습
def train(args):
    seed_everything(args.seed)
    dev = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")
    print("Device:", dev)

    # 전체 CSV 로드
    df = pd.read_csv(args.data_path, encoding="utf-8-sig")

    tok = GPT2Tokenizer.from_pretrained(args.model_size)
    tok.pad_token = tok.eos_token

    model = GPT2Explainer(args).to(dev)
    optim = torch.optim.AdamW(model.parameters(), lr=args.lr)

    best_ppl = 1e9  # 낮을수록 좋음

    # ----------- Curriculum: grade 2 다음 grade 3 ----------
    for grade in [2, 3]:
        print(f"\n=====  Curriculum Stage: Grade {grade}  =====")
        train_df = df[(df.split == "train") & (df.grade == grade)]
        valid_df = df[(df.split == "valid") & (df.grade == grade)]
        if len(train_df) == 0:
            print(f"  (Grade {grade} 데이터가 없습니다, 스킵)")
            continue

        tr_ds = ExplDataset(train_df, tok, args.max_length)
        va_ds = ExplDataset(valid_df, tok, args.max_length)
        tr_ld = DataLoader(tr_ds, batch_size=args.batch_size, shuffle=True,
                           collate_fn=tr_ds.collate_fn)
        va_ld = DataLoader(va_ds, batch_size=args.batch_size, shuffle=False,
                           collate_fn=va_ds.collate_fn)

        # -------- epoch loop --------
        for ep in range(1, args.epochs + 1):
            model.train(); ep_loss = 0
            for bt in tqdm(tr_ld, disable=TQDM_DISABLE,
                           desc=f"(G{grade}) Epoch {ep}"):
                optim.zero_grad()
                loss, _ = model(bt["input_ids"].to(dev),
                                bt["attention_mask"].to(dev),
                                bt["labels"].to(dev))
                loss.backward(); optim.step()
                ep_loss += loss.item()
            print(f"   >> train_loss={ep_loss/len(tr_ld):.4f}")

            ppl = evaluate(va_ld, model, dev)
            print(f"   >> valid PPL={ppl:.3f}")

            if ppl < best_ppl:
                best_ppl = ppl
                save(model, optim, args)

def save(m, o, args):
    os.makedirs(os.path.dirname(args.save_path), exist_ok=True)
    torch.save({"model": m.state_dict(),
                "optim": o.state_dict(),
                "args":  args}, args.save_path)
    print("  ** Best model saved →", args.save_path)

# ───────────────────────────── 인자
def get_args():
    p = argparse.ArgumentParser()
    p.add_argument("--data_path", type=str,
                   default="/content/drive/MyDrive/CSEG321/dataset/explanation_all_options.csv")
    p.add_argument("--model_size", type=str, default="gpt2",
                   choices=["gpt2", "gpt2-medium", "gpt2-large"])
    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--max_length", type=int, default=512)
    p.add_argument("--lr", type=float, default=5e-5)
    p.add_argument("--epochs", type=int, default=50)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--use_gpu", action="store_true")
    args = p.parse_args([])
    if torch.cuda.is_available(): args.use_gpu = True
    # hidden dim 등
    if args.model_size == "gpt2":
        args.d, args.l, args.num_heads = 768, 12, 12
    elif args.model_size == "gpt2-medium":
        args.d, args.l, args.num_heads = 1024, 24, 16
    else:
        args.d, args.l, args.num_heads = 1280, 36, 20

    args.save_path = "/content/drive/MyDrive/CSEG321/models/expl_all_curr_50.pt"
    return args

# ───────────────────────────── main
if __name__ == "__main__":
    seed_everything()
    train(get_args())

Device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

   >> train_loss=0.3758
   >> valid PPL=14588.724


(G3) Epoch 49: 100%|██████████| 29/29 [00:04<00:00,  6.89it/s]


   >> train_loss=0.3530
   >> valid PPL=14753.851


(G3) Epoch 50: 100%|██████████| 29/29 [00:04<00:00,  6.91it/s]


   >> train_loss=0.3396
   >> valid PPL=15492.765


In [4]:
"""
train_expl_all_curr.py
GPT-2 해설 생성 (모든 선택지 포함) + Curriculum Learning (grade 2 ➜ grade 3)
"""

import argparse, os, random, numpy as np, pandas as pd, torch
import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model   # 반드시 여러분의 커스텀 GPT2Model

TQDM_DISABLE = False  # True면 TQDM bar 숨김

# ───────────────────────────── utils
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False

# ───────────────────────────── 모델
class GPT2Explainer(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.backbone = GPT2Model.from_pretrained(
            model=args.model_size, d=args.d, l=args.l, num_heads=args.num_heads
        )
        vocab = GPT2Tokenizer.from_pretrained(args.model_size).vocab_size
        self.lm_head = nn.Linear(args.d, vocab, bias=False)
        for p in self.parameters(): p.requires_grad = True

    def forward(self, ids, mask, labels=None):
        h = self.backbone(ids, attention_mask=mask)["last_hidden_state"]
        logits = self.lm_head(h)                       # (B, L, V)
        if labels is None:
            return logits
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=-100
        )
        return loss, logits

# ───────────────────────────── 데이터셋
class ExplDataset(Dataset):
    def __init__(self, df, tok, max_len=512):
        self.df, self.tok, self.max_len = df.reset_index(drop=True), tok, max_len
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        return r["input"], r["target"]
    def collate_fn(self, batch):
        ins, tgs = zip(*batch)
        enc_in  = self.tok(list(ins), padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        enc_out = self.tok(list(tgs), padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        labels = enc_out.input_ids.clone()
        labels[enc_out.attention_mask == 0] = -100
        return {"input_ids": enc_in.input_ids,
                "attention_mask": enc_in.attention_mask,
                "labels": labels}

# ───────────────────────────── 평가
def evaluate(loader, model, dev):
    model.eval(); tot_loss = tok_cnt = 0
    with torch.no_grad():
        for bt in loader:
            ids  = bt["input_ids"].to(dev)
            mask = bt["attention_mask"].to(dev)
            lbls = bt["labels"].to(dev)
            loss, _ = model(ids, mask, lbls)
            num_tok = (lbls != -100).sum().item()
            tot_loss += loss.item() * num_tok
            tok_cnt  += num_tok
    ppl = np.exp(tot_loss / max(tok_cnt, 1))
    return ppl

# ───────────────────────────── 학습
def train(args):
    seed_everything(args.seed)
    dev = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")
    print("Device:", dev)

    # 전체 CSV 로드
    df = pd.read_csv(args.data_path, encoding="utf-8-sig")

    tok = GPT2Tokenizer.from_pretrained(args.model_size)
    tok.pad_token = tok.eos_token

    model = GPT2Explainer(args).to(dev)
    optim = torch.optim.AdamW(model.parameters(), lr=args.lr)

    best_ppl = 1e9  # 낮을수록 좋음

    # ----------- Curriculum: grade 2 다음 grade 3 ----------
    for grade in [2, 3]:
        print(f"\n=====  Curriculum Stage: Grade {grade}  =====")
        train_df = df[(df.split == "train") & (df.grade == grade)]
        valid_df = df[(df.split == "valid") & (df.grade == grade)]
        if len(train_df) == 0:
            print(f"  (Grade {grade} 데이터가 없습니다, 스킵)")
            continue

        tr_ds = ExplDataset(train_df, tok, args.max_length)
        va_ds = ExplDataset(valid_df, tok, args.max_length)
        tr_ld = DataLoader(tr_ds, batch_size=args.batch_size, shuffle=True,
                           collate_fn=tr_ds.collate_fn)
        va_ld = DataLoader(va_ds, batch_size=args.batch_size, shuffle=False,
                           collate_fn=va_ds.collate_fn)

        # -------- epoch loop --------
        for ep in range(1, args.epochs + 1):
            model.train(); ep_loss = 0
            for bt in tqdm(tr_ld, disable=TQDM_DISABLE,
                           desc=f"(G{grade}) Epoch {ep}"):
                optim.zero_grad()
                loss, _ = model(bt["input_ids"].to(dev),
                                bt["attention_mask"].to(dev),
                                bt["labels"].to(dev))
                loss.backward(); optim.step()
                ep_loss += loss.item()
            print(f"   >> train_loss={ep_loss/len(tr_ld):.4f}")

            ppl = evaluate(va_ld, model, dev)
            print(f"   >> valid PPL={ppl:.3f}")

            if ppl < best_ppl:
                best_ppl = ppl
                save(model, optim, args)

def save(m, o, args):
    os.makedirs(os.path.dirname(args.save_path), exist_ok=True)
    torch.save({"model": m.state_dict(),
                "optim": o.state_dict(),
                "args":  args}, args.save_path)
    print("  ** Best model saved →", args.save_path)

# ───────────────────────────── 인자
def get_args():
    p = argparse.ArgumentParser()
    p.add_argument("--data_path", type=str,
                   default="/content/drive/MyDrive/CSEG321/dataset/explanation_only_answer.csv")
    p.add_argument("--model_size", type=str, default="gpt2",
                   choices=["gpt2", "gpt2-medium", "gpt2-large"])
    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--max_length", type=int, default=512)
    p.add_argument("--lr", type=float, default=5e-5)
    p.add_argument("--epochs", type=int, default=50)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--use_gpu", action="store_true")
    args = p.parse_args([])
    if torch.cuda.is_available(): args.use_gpu = True
    # hidden dim 등
    if args.model_size == "gpt2":
        args.d, args.l, args.num_heads = 768, 12, 12
    elif args.model_size == "gpt2-medium":
        args.d, args.l, args.num_heads = 1024, 24, 16
    else:
        args.d, args.l, args.num_heads = 1280, 36, 20

    args.save_path = "/content/drive/MyDrive/CSEG321/models/expl_only_answer_curr_50.pt"
    return args

# ───────────────────────────── main
if __name__ == "__main__":
    seed_everything()
    train(get_args())

Device: cuda

=====  Curriculum Stage: Grade 2  =====


(G2) Epoch 1: 100%|██████████| 7/7 [00:01<00:00,  6.34it/s]


   >> train_loss=14.2505
   >> valid PPL=187714.827
  ** Best model saved → /content/drive/MyDrive/CSEG321/models/expl_only_answer_curr_50.pt


(G2) Epoch 2: 100%|██████████| 7/7 [00:01<00:00,  6.68it/s]


   >> train_loss=11.2944
   >> valid PPL=63569.182
  ** Best model saved → /content/drive/MyDrive/CSEG321/models/expl_only_answer_curr_50.pt


...

   >> train_loss=0.3512
   >> valid PPL=12619.583


(G3) Epoch 49: 100%|██████████| 29/29 [00:04<00:00,  6.91it/s]


   >> train_loss=0.3332
   >> valid PPL=12994.642


(G3) Epoch 50: 100%|██████████| 29/29 [00:04<00:00,  6.95it/s]


   >> train_loss=0.3170
   >> valid PPL=13259.361
